In [1]:
import os
import re
from pathlib import Path
import numpy as np
import spacy
import nltk

nltk.download('wordnet')
nltk.download('punkt')

nlp = spacy.load("en_core_web_md")

import string
from sklearn.feature_extraction.text import TfidfVectorizer as ti
import pandas as pd
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer


class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()

    def __call__(self, articles):
        return [self.wnl.lemmatize(t, pos='v') for t in word_tokenize(articles)]

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/arpanjeetsandhu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/arpanjeetsandhu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
txt = Path("/Users/arpanjeetsandhu/Desktop/Hadoop/application_1445087491445_0005/container_1445087491445_0005_01_000001.log").read_text()
print(txt)

2015-10-17 21:47:48,288 INFO [main] org.apache.hadoop.mapreduce.v2.app.MRAppMaster: Created MRAppMaster for application appattempt_1445087491445_0005_000001
2015-10-17 21:47:49,078 INFO [main] org.apache.hadoop.mapreduce.v2.app.MRAppMaster: Executing with tokens:
2015-10-17 21:47:49,078 INFO [main] org.apache.hadoop.mapreduce.v2.app.MRAppMaster: Kind: YARN_AM_RM_TOKEN, Service: , Ident: (appAttemptId { application_id { id: 5 cluster_timestamp: 1445087491445 } attemptId: 1 } keyId: -1547346236)
2015-10-17 21:47:49,370 INFO [main] org.apache.hadoop.mapreduce.v2.app.MRAppMaster: Using mapred newApiCommitter.
2015-10-17 21:47:50,570 INFO [main] org.apache.hadoop.mapreduce.v2.app.MRAppMaster: OutputCommitter set in config null
2015-10-17 21:47:50,635 INFO [main] org.apache.hadoop.mapreduce.v2.app.MRAppMaster: OutputCommitter is org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter
2015-10-17 21:47:50,677 INFO [main] org.apache.hadoop.yarn.event.AsyncDispatcher: Registering class org.ap

In [3]:
regex = '(([0-9]+(-[0-9]+)+) ([0-9]+(:[0-9]+)+),[0-9]+\s[a-zA-z]+ \[[^\]]*] [a-zA-Z0-9.a-zA-Z0-9.a-zA-Z0-9]+.:)'
txt = re.sub(regex, "++++++", txt)
txt = re.sub(r'[0-9]+', "number", txt);
txt = re.sub("\n", "", txt)
print(txt)

++++++ Created MRAppMaster for application appattempt_number_number_number++++++ Executing with tokens:++++++ Kind: YARN_AM_RM_TOKEN, Service: , Ident: (appAttemptId { application_id { id: number cluster_timestamp: number } attemptId: number } keyId: -number)++++++ Using mapred newApiCommitter.++++++ OutputCommitter set in config null++++++ OutputCommitter is org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter++++++ Registering class org.apache.hadoop.mapreduce.jobhistory.EventType for class org.apache.hadoop.mapreduce.jobhistory.JobHistoryEventHandler++++++ Registering class org.apache.hadoop.mapreduce.vnumber.app.job.event.JobEventType for class org.apache.hadoop.mapreduce.vnumber.app.MRAppMaster$JobEventDispatcher++++++ Registering class org.apache.hadoop.mapreduce.vnumber.app.job.event.TaskEventType for class org.apache.hadoop.mapreduce.vnumber.app.MRAppMaster$TaskEventDispatcher++++++ Registering class org.apache.hadoop.mapreduce.vnumber.app.job.event.TaskAttemptEventType f

In [4]:
logs_list = re.split(r'[++++++]', txt);
logs_list

['',
 '',
 '',
 '',
 '',
 '',
 ' Created MRAppMaster for application appattempt_number_number_number',
 '',
 '',
 '',
 '',
 '',
 ' Executing with tokens:',
 '',
 '',
 '',
 '',
 '',
 ' Kind: YARN_AM_RM_TOKEN, Service: , Ident: (appAttemptId { application_id { id: number cluster_timestamp: number } attemptId: number } keyId: -number)',
 '',
 '',
 '',
 '',
 '',
 ' Using mapred newApiCommitter.',
 '',
 '',
 '',
 '',
 '',
 ' OutputCommitter set in config null',
 '',
 '',
 '',
 '',
 '',
 ' OutputCommitter is org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter',
 '',
 '',
 '',
 '',
 '',
 ' Registering class org.apache.hadoop.mapreduce.jobhistory.EventType for class org.apache.hadoop.mapreduce.jobhistory.JobHistoryEventHandler',
 '',
 '',
 '',
 '',
 '',
 ' Registering class org.apache.hadoop.mapreduce.vnumber.app.job.event.JobEventType for class org.apache.hadoop.mapreduce.vnumber.app.MRAppMaster$JobEventDispatcher',
 '',
 '',
 '',
 '',
 '',
 ' Registering class org.apache.hadoop.mapred

In [5]:
import os
import re
from pathlib import Path
import numpy as np


class LogPreprocessing:
    def __init__(self, path):
        self.path = path;
        self.regex = '(([0-9]+(-[0-9]+)+) ([0-9]+(:[0-9]+)+),[0-9]+\s[a-zA-z]+ \[[^\]]*] [a-zA-Z0-9.a-zA-Z0-9.a-zA-Z0-9]+.:)'
        self.normal_log_file_names = ['application_1445087491445_0005', 'application_1445087491445_0007',
                                      'application_1445175094696_0005',
                                      'application_1445062781478_0011',
                                      'application_1445062781478_0016',
                                      'application_1445062781478_0019',
                                      'application_1445076437777_0002',
                                      'application_1445076437777_0005',
                                      'application_1445144423722_0021',
                                      'application_1445144423722_0024',
                                      'application_1445182159119_0012'
                                      ]

    def isLog(self, x):
        if len(x) > 0:
            return True;
        else:
            return False

    def get_log_text(self, file, folder):
        txt = Path(self.path + "/" + folder + "/" + file).read_text()
        txt = re.sub(self.regex, "splitonthis", txt);
        txt = re.sub(r'[0-9]+', "number", txt);
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        txt = regex.sub(' ', txt)
        txt = re.sub("\n", "", txt)
        logs_list = re.split(r'splitonthis', txt);
        return logs_list

    def get_normal_log(self):
        self.logs = []
        self.all_normal_logs= []
        dir = os.listdir(self.path)
        first = True;
        for folder in dir:
            if folder != '.DS_Store' and folder != 'abnormal_label.txt' and self.normal_log_file_names.__contains__(folder):
                textFiles = os.listdir(self.path + "/" + folder);
                for file in textFiles:
                    if file != '.DS_Store':
                        logs_list = self.get_log_text(file, folder)
                        final_list = [];
                        for log in filter(self.isLog, logs_list):
                            if not self.all_normal_logs.__contains__(log):
                                final_list.append(log);
                                self.all_normal_logs.append(log);
                        nd_array = np.column_stack((final_list, np.ones(len(final_list))))
                        if first:
                            self.logs = nd_array
                            first = False
                        else:
                            self.logs = np.append(self.logs, nd_array, axis=0);


    def all_log_data(self):
        all_log = []
        dir = os.listdir(self.path);
        for folder in dir:
            if folder != '.DS_Store' and folder != 'abnormal_label.txt' and not self.normal_log_file_names.__contains__(folder):
                textFiles = os.listdir(self.path + "/" + folder);
                for file in textFiles:
                    if file != '.DS_Store':
                        logs_list = self.get_log_text(file, folder)
                        final_list = []
                        label = []
                        for log in filter(self.isLog, logs_list):
                            if not self.all_normal_logs.__contains__(log):
                                if not all_log.__contains__(log):
                                    final_list.append(log)
                                    label.append(0)
                                    all_log.append(log)
                        nd_array = np.column_stack((final_list, label))
                        self.logs = np.append(self.logs, nd_array, axis=0)


logPreprocessing = LogPreprocessing("/Users/arpanjeetsandhu/Desktop/Hadoop")
logPreprocessing.get_normal_log()
logPreprocessing.all_log_data()

In [6]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

In [7]:
logPreprocessing.logs

array([[' loaded properties from hadoop metricsnumber properties', '1.0'],
       [' Scheduled snapshot period at number second s  ', '1.0'],
       [' MapTask metrics system started', '1.0'],
       [' Executing with tokens ', '1.0'],
       [' Kind  mapreduce job  Service  job number number  Ident   org apache hadoop mapreduce security token JobTokenIdentifier numberfnumberebnumberbnumber ',
        '1.0'],
       [' Sleeping for numberms before retrying again  Got null now ',
        '1.0'],
       [' mapreduce cluster local dir for child   tmp hadoop msrabi nm local dir usercache msrabi appcache application number number',
        '1.0'],
       [' session id is deprecated  Instead  use dfs metrics session id',
        '1.0'],
       [' ProcfsBasedProcessTree currently is supported only on Linux ',
        '1.0'],
       ['  Using ResourceCalculatorProcessTree   org apache hadoop yarn util WindowsBasedProcessTree numberadnumberf',
        '1.0'],
       [' Processing split  hdfs   

In [8]:
similarity_arr = []
number_of_word = []
# number_of_similar logs
# number of words in logs
doc_1 = nlp(str(logPreprocessing.logs[1][0]))
for log in logPreprocessing.logs:
    word = str(log[0])
    number_of_word.append(len(word.split(" ")))
    doc_2 = nlp(word)
    a = doc_1.similarity(doc_2)
    similarity_arr.append(a)

unique_number = list(set(similarity_arr))
dict = {}
for num in unique_number:
    count = 0
    for similarity in similarity_arr:
        if num == similarity:
            count = count + 1
    dict[num] = count

final_arr = []

for i in similarity_arr:
    final_arr.append(dict.get(i))

logPreprocessing.logs = np.c_[logPreprocessing.logs, final_arr]
logPreprocessing.logs = np.c_[logPreprocessing.logs, number_of_word]

/var/folders/lb/v9prt0qs7rn8t5y1kvn44n440000gn/T/ipykernel_49168/2556049537.py:10: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  a = doc_1.similarity(doc_2)


In [9]:
logPreprocessing.logs

array([[' loaded properties from hadoop metricsnumber properties', '1.0',
        '1', '7'],
       [' Scheduled snapshot period at number second s  ', '1.0', '1',
        '10'],
       [' MapTask metrics system started', '1.0', '3', '5'],
       [' Executing with tokens ', '1.0', '1', '5'],
       [' Kind  mapreduce job  Service  job number number  Ident   org apache hadoop mapreduce security token JobTokenIdentifier numberfnumberebnumberbnumber ',
        '1.0', '66', '24'],
       [' Sleeping for numberms before retrying again  Got null now ',
        '1.0', '1', '12'],
       [' mapreduce cluster local dir for child   tmp hadoop msrabi nm local dir usercache msrabi appcache application number number',
        '1.0', '1', '21'],
       [' session id is deprecated  Instead  use dfs metrics session id',
        '1.0', '1', '13'],
       [' ProcfsBasedProcessTree currently is supported only on Linux ',
        '1.0', '1', '9'],
       ['  Using ResourceCalculatorProcessTree   org apach

In [10]:
# TF_IDF MATRIX
log_list = list(logPreprocessing.logs[0:len(logPreprocessing.logs), 0])



tokenizer = LemmaTokenizer()
tf_idf = ti(stop_words='english', analyzer="word", tokenizer=tokenizer)
tf_idf_matrix = tf_idf.fit_transform(log_list)
tfidf_df = pd.DataFrame(tf_idf_matrix.toarray())

print("done")


/Users/arpanjeetsandhu/opt/anaconda3/envs/untitled/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['make'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


done


In [ ]:
# "MapTask metrics system started",
#  "ReduceTask metrics system started",
#  "MRAppMaster metrics system started",
# "MapTask metrics system stopped",
#  "ReduceTask metrics system stopped"